# Load libraries and Themes

In [1]:
suppressPackageStartupMessages({
        suppressWarnings({
        library(Seurat)
        library(SoupX)
        library(ggplot2)
        library(tidyverse)
        library(scDblFinder)
        library(harmony)
        library(SeuratDisk)
        library(SingleCellExperiment)
        library(dplyr)
        library(ggpubr)
        library(pals)
        library(viridis)
        library(scCustomize)
            })})

In [ ]:
setwd("/media/daten/dmeral/scseq_analysis/2024_LA_CTRL_ALDO")

In [ ]:
set.seed(1234)

# SoupX (remove ambient signal)

In [ ]:
ContaminationFraction <- 0.25

In [ ]:
samples <- c("LA_2109", "LA_2111", "LA_2112", "LA_2115")

In [ ]:
# Simple pre-process

mad_outlier <- function(sobj, metric, nmads){
  M <- sobj@meta.data[[metric]]
  median_M <- median(M, na.rm = TRUE)
  mad_M <- mad(M, na.rm = TRUE)
  outlier <- (M < (median_M - nmads * mad_M)) | (M > (median_M + nmads * mad_M))
  return(outlier)
}

    # Load in filtered cellranger outs
filter_mad_outliers <- function(sample_id){
  path <- paste0("/media/daten/dmeral/scseq/2024_LA_CTRL_ALDO/", sample_id, "/outs/filtered_feature_bc_matrix/")
  sobj <- Read10X(data.dir = path)
  sobj <- CreateSeuratObject(counts = sobj, min.cells = 0, min.features = 200)
  sobj$sample_id <- sample_id
  
  # add QC metrics
  sobj$log1p_total_counts <- log1p(sobj@meta.data$nCount_RNA)
  sobj$log1p_n_genes_by_counts <- log1p(sobj@meta.data$nFeature_RNA)
  sobj[["percent.mt"]] <- PercentageFeatureSet(sobj, pattern = "^mt-")
  
  # find outliers and subset
  bool_vector <- !mad_outlier(sobj, "log1p_total_counts", 5) & !mad_outlier(sobj, "log1p_n_genes_by_counts", 5) & !mad_outlier(sobj, 'percent.mt', 5)
  sobj <- subset(sobj, cells = which(bool_vector))
  
  return(sobj)
}

In [ ]:
data_list <- sapply(samples, filter_mad_outliers)

In [ ]:
# Basic seurat normalization and clustering
get_soup_groups <- function(sobj){
  sobj <- NormalizeData(sobj, verbose = FALSE)
  sobj <- FindVariableFeatures(object = sobj, nfeatures = 2000, verbose = FALSE, selection.method = "vst")
  sobj <- ScaleData(sobj, verbose = FALSE)
  sobj <- RunPCA(sobj, npcs = 35, verbose = FALSE)
  sobj <- FindNeighbors(sobj, dims = 1:35, verbose = FALSE)
  sobj <- FindClusters(sobj, resolution = 0.5, verbose = FALSE)
  
  return(sobj@meta.data[["seurat_clusters"]])
}

In [ ]:
add_soup_groups <- function(sobj){
  sobj$soup_group <- get_soup_groups(sobj)
  return(sobj)
}

data_list <- sapply(data_list, add_soup_groups)

In [ ]:
make_soup <- function(sobj){
  sample_id <- as.character(sobj$sample_id[1])  # e.g., LA_2109
  path <- paste0("/media/daten/dmeral/scseq/2024_LA_CTRL_ALDO/", sample_id, "/outs/raw_feature_bc_matrix/")
  raw <- Read10X(data.dir = path)

  sc <- SoupChannel(raw, GetAssayData(sobj, layer = "counts"))
  sc <- setClusters(sc, sobj$soup_group)
  sc <- setContaminationFraction(sc, ContaminationFraction)
  out <- adjustCounts(sc, roundToInt = TRUE)
  
  # Optional: keep original counts
  sobj[["original.counts"]] <- CreateAssayObject(counts = GetAssayData(sobj, layer = "counts"))
  
  # Set adjusted counts back to the RNA assay
  sobj <- SetAssayData(sobj, layer = "counts", new.data = out)
  
  return(sobj)
}

In [ ]:
suppressWarnings({
    data_list <- sapply(data_list, make_soup)
})

In [ ]:
# Check if  correct number of reads were removed by SoupX
# Create a data frame to store results
results <- data.frame(Sample = samples, Counts_Before = NA, Counts_After = NA, Fraction_Left = NA)

# Iterate through each sample
for (i in seq_along(samples)) {
  sample_name <- samples[i]
  data_obj <- data_list[[i]] 

  # Check if the assay exists
  if ("original.counts" %in% names(data_obj) && "RNA" %in% names(data_obj)) {
    counts_before <- sum(GetAssayData(data_obj, assay = "original.counts", layer = "counts"))
    counts_after <- sum(GetAssayData(data_obj, assay = "RNA", layer = "counts"))

    # Store results in the data frame
    results[i, "Counts_Before"] <- counts_before
    results[i, "Counts_After"] <- counts_after
    
    # Calculate percent left if counts_before is not zero
    if (!is.na(counts_before) && counts_before > 0) {
      results[i, "Fraction_Left"] <- counts_after / counts_before
    }
  } else {
    message(paste("Assay not found for sample:", sample_name))
  }
}

# Print 
print(results)

# scDblFinder (remove doublets)

In [ ]:
saveRDS(data_list, paste0("seurat_objects/rds_data_list_", ContaminationFraction, ".rds"))

In [ ]:
data_list <- readRDS(paste0("seurat_objects/rds_data_list_", ContaminationFraction, ".rds"))

In [ ]:
#Normalize/ Center and Scale Matrix
data_list_scale <- lapply(data_list, function(obj.seu) {
   obj.seu <- NormalizeData(obj.seu, verbose = FALSE)
   obj.seu <- ScaleData(obj.seu, verbose = FALSE)
   return(obj.seu)
})

In [ ]:
# Convert to SingleCellExperiment
obj_sce <- lapply(data_list_scale, function(obj.seu){
    as.SingleCellExperiment(obj.seu, assay = "RNA")
})

In [ ]:
#Run scDblFinder
obj_scDblFinder <- lapply(obj_sce, function(expression_matrix){
   scDblFinder(expression_matrix)
})

In [ ]:
# Convert to Seurat objet
obj_seu <- lapply(obj_scDblFinder, function(sce){
    as.Seurat(x = sce, 
              counts = "counts", 
              data = "logcounts")
})

In [ ]:
table_singlets <- lapply(obj_seu, function(obj){
    table(obj$scDblFinder.class)
})

print(table_singlets)

# Add meta.data, filter singlets and merge

In [ ]:
meta.data_tab <- matrix(c(rep(1, each = 4), 
                          "CTRL", "ALDO", "ALDO", "CTRL",
                          "f", "f", "m", "m",
                          rep(1, each = 4), 
                          rep("LA", each = 4), 
                          1:4, 
                          rep(ContaminationFraction, each = 4)),
                          ncol = 4, byrow = TRUE)

rownames(meta.data_tab) <- c("replicate", "treatment", "sex", "batch", "chamber", "unique", "ContaminationFraction")
colnames(meta.data_tab) <- samples

meta.data_tab <- as.table(meta.data_tab)
meta.data_tab

In [ ]:
# Adding metadata to Seurat objects with alignment
obj_seu_newmeta <- lapply(samples, function(nam) {
    # Retrieve the Seurat object for the current sample
    scobj <- obj_seu[[nam]]
    
    # Extract the corresponding metadata and convert it to a data frame
    meta <- meta.data_tab[, nam]
    
    # Ensure the metadata is in the right format
    meta_df <- as.data.frame(matrix(rep(meta, ncol(scobj)), nrow = ncol(scobj), byrow = TRUE))
    colnames(meta_df) <- rownames(meta.data_tab)  # Set column names to match the metadata structure
    
    # Add metadata to the Seurat object
    scobj <- AddMetaData(scobj, metadata = meta_df)
    
    return(scobj)  # Return the modified Seurat object
})

# Assign names to the new list
names(obj_seu_newmeta) <- samples

In [ ]:
# Merge in one object
obj_seu_merge <- Merge_Seurat_List(obj_seu_newmeta, add.cell.ids = c(1:4),  merge.data = TRUE, project = "LA_C57Bl6_Aldosterone")

In [ ]:
SaveH5Seurat(obj_seu_merge, paste0("seurat_objects/setContaminationFraction/setContaminationFraction_", ContaminationFraction, "_dbl"))

# Harmony (integration)

**run Harmony, UMAP using Harmony embeddings instead of PCA and visualize**
Resolution can dramatically effect your clustering. While the overall shape of your tSNE or UMAP will not change the number of clusterings will be altered. 
This is something you need to dynamically adjust and examine as it relates to your dataset. To high a resolution and you end up with ill defined clusters 
which lack unique markers. To low and you end up with clusters that are too large and you may miss meaningful info.

In [ ]:
#Filter Singlets
obj_seu_merge_singlet <- subset(obj_seu_merge, scDblFinder.class == "singlet")

In [ ]:
# switch default assay to RNA
DefaultAssay(object = obj_seu_merge_singlet) <- "RNA"

In [ ]:
obj <- subset(obj_seu_merge_singlet, subset = nFeature_RNA > 300 & nFeature_RNA < 5000 & 
                        nCount_RNA > 500 & nCount_RNA < 15000 &
                        percent.mt < 5)

obj <- NormalizeData(obj, verbose = FALSE)
obj <- FindVariableFeatures(obj, verbose = FALSE)
obj <- ScaleData(obj, verbose = FALSE)
obj <- RunPCA(obj, verbose = FALSE)

In [ ]:
# Integrate "sex"
obj <- obj %>%
  RunHarmony(group.by.vars = c("sex"), plot_convergence = FALSE, assay.use = "RNA", verbose = FALSE)

obj_embed <- Embeddings(obj, "harmony")

In [ ]:
obj_harmony <- obj %>%
  RunUMAP(reduction = "harmony", dims = 1:35, verbose = FALSE) %>%
  FindNeighbors(reduction = "harmony", dims = 1:35, verbose = FALSE) %>%
  FindClusters(resolution = 0.15)

In [ ]:
SaveH5Seurat(obj_harmony, paste0("seurat_objects/setContaminationFraction/setContaminationFraction_", ContaminationFraction, "_harmony_sgl"))

# FindAllMarkers

In [ ]:
# used assay RNA
all.markers <- FindAllMarkers(obj_harmony, only.pos = TRUE, min.pct = 0.50, logfc.threshold = 0.58)
all.markers %>%
  group_by("seurat_clusters") %>%
  slice_max(n= 10, order_by = avg_log2FC)

write.csv(all.markers, paste0("setContaminationFraction/all.markers_wilcox_setContaminationFraction_", ContaminationFraction, "_harmony_sgl.csv"))

In [ ]:
# Initialize a list to store marker results for each sample
marker_results <- list()

for (sample in samples) {
    obj_sample <- subset(obj_harmony, subset = sample_id == sample)
    obj_sample <- subset(obj_sample, subset = seurat_clusters %in% 0:5)
    all.markers <- FindAllMarkers(obj_sample, 
                                only.pos = TRUE, 
                                min.pct = 0.75, 
                                logfc.threshold = 0.58)

    top.markers <- all.markers %>%
    group_by(cluster) %>%
    slice_max(n = 10, order_by = avg_log2FC)
  
  write.csv(top.markers, paste0("setContaminationFraction/all.markers_wilcox_setContaminationFraction_", 
                                ContaminationFraction, "_", sample, "_harmony_sgl.csv"))
  
  marker_results[[sample]] <- top.markers
}